## Advanced Slicing Practice (with Solutions)

**Topic:** NumPy slicing (1D + 2D), views vs copies, slicing assignment, negative steps, broadcasting.

**Best practices used:**
- Exercises are *self-contained*.
- Solutions include `assert` checks.
- Prefer vectorized slicing over loops.
- Clearly separate *Problem* vs *Solution*.


In [1]:
import numpy as np
from numpy.lib.stride_tricks import sliding_window_view

### Exercise 1 — Prove slice is a view (and how to break the link)

**Problem**
1. Create `a = np.arange(12)`.
2. Create `v = a[2:10:3]`.
3. Modify `v[1]` and show it changes the corresponding element of `a`.
4. Create `c = a[2:10:3].copy()` and show modifying `c` does **not** affect `a`.
5. Use `np.shares_memory` to validate view vs copy.

In [2]:
# --- Solution (Exercise 1) ---
a = np.arange(12)
v = a[2:10:3]  # indices: 2,5,8

assert np.shares_memory(a, v)

a_before = a.copy()
v[1] = 999

print("a before:", a_before)
print("a after :", a)
print("v      :", v)

assert a[5] == 999, "v[1] maps to a[5] (2,5,8)."

c = a[2:10:3].copy()
assert not np.shares_memory(a, c)

a_before2 = a.copy()
c[0] = -123

print("\na unchanged after editing copy?", np.array_equal(a, a_before2))
assert np.array_equal(a, a_before2)

a before: [ 0  1  2  3  4  5  6  7  8  9 10 11]
a after : [  0   1   2   3   4 999   6   7   8   9  10  11]
v      : [  2 999   8]

a unchanged after editing copy? True


### Exercise 2 — 2D slicing with negative steps

**Problem**
1. Create `M = np.arange(36).reshape(6, 6)`.
2. Create a view `B` containing:
   - rows `4` down to `1` (inclusive), stepping by `-1`
   - columns `5` down to `0` (inclusive), stepping by `-2`
3. Verify `B.shape == (4, 3)`.
4. Prove that editing `B` edits `M`.

In [3]:
# --- Solution (Exercise 2) ---
M = np.arange(36).reshape(6, 6)

# rows: 4,3,2,1
# cols: 5,3,1  (use 5::-2 to get step -2 including index 1)
B = M[4:0:-1, 5::-2]

print("M:\n", M)
print("\nB:\n", B)
print("B.shape:", B.shape)

assert B.shape == (4, 3)
assert np.shares_memory(M, B)

old = M[4, 5]
B[0, 0] = 10_000
assert M[4, 5] == 10_000

M:
 [[ 0  1  2  3  4  5]
 [ 6  7  8  9 10 11]
 [12 13 14 15 16 17]
 [18 19 20 21 22 23]
 [24 25 26 27 28 29]
 [30 31 32 33 34 35]]

B:
 [[29 27 25]
 [23 21 19]
 [17 15 13]
 [11  9  7]]
B.shape: (4, 3)


### Exercise 3 — Double border frame (slicing assignment only)

**Problem**
Create a `10x10` int array of zeros. Using **only slicing assignments** (no loops, no fancy indexing):
- set the outer border to `1`
- set the inner border (border of the central `8x8` region) to `2`

The result should look like a double frame.

In [4]:
# --- Solution (Exercise 3) ---
A = np.zeros((10, 10), dtype=int)

# Outer border
A[0, :] = 1
A[-1, :] = 1
A[:, 0] = 1
A[:, -1] = 1

# Inner border at indices 1 and -2
A[1, 1:-1] = 2
A[-2, 1:-1] = 2
A[1:-1, 1] = 2
A[1:-1, -2] = 2

print(A)

# Checks
assert (A[0, :] == 1).all() and (A[-1, :] == 1).all()
assert (A[:, 0] == 1).all() and (A[:, -1] == 1).all()
assert (A[1, 1:-1] == 2).all() and (A[-2, 1:-1] == 2).all()
assert (A[1:-1, 1] == 2).all() and (A[1:-1, -2] == 2).all()

[[1 1 1 1 1 1 1 1 1 1]
 [1 2 2 2 2 2 2 2 2 1]
 [1 2 0 0 0 0 0 0 2 1]
 [1 2 0 0 0 0 0 0 2 1]
 [1 2 0 0 0 0 0 0 2 1]
 [1 2 0 0 0 0 0 0 2 1]
 [1 2 0 0 0 0 0 0 2 1]
 [1 2 0 0 0 0 0 0 2 1]
 [1 2 2 2 2 2 2 2 2 1]
 [1 1 1 1 1 1 1 1 1 1]]


### Exercise 4 — Checkerboard (no loops)

**Problem**
Create an `8x8` checkerboard array `C` with values 0/1 such that:
- top-left is `0`
- adjacent cells differ

Use slicing with steps.

In [5]:
# --- Solution (Exercise 4) ---
C = np.zeros((8, 8), dtype=int)
C[0::2, 1::2] = 1
C[1::2, 0::2] = 1

print(C)
assert C.sum() == 32

# A quick structural check: any 2x2 block must contain two 0s and two 1s
block = C[2:4, 3:5]
assert sorted(block.ravel().tolist()) == [0, 0, 1, 1]

[[0 1 0 1 0 1 0 1]
 [1 0 1 0 1 0 1 0]
 [0 1 0 1 0 1 0 1]
 [1 0 1 0 1 0 1 0]
 [0 1 0 1 0 1 0 1]
 [1 0 1 0 1 0 1 0]
 [0 1 0 1 0 1 0 1]
 [1 0 1 0 1 0 1 0]]


### Exercise 5 — Broadcasting into a slice + in-place update

**Problem**
Let `X` be a `5x7` zero matrix.
1. Set columns 2..5 (inclusive) to `[10, 20, 30, 40]` for every row.
2. Then add `+5` (in place) to rows 1..3 (inclusive) within that same block.

No loops.

In [6]:
# --- Solution (Exercise 5) ---
X = np.zeros((5, 7), dtype=int)

X[:, 2:6] = [10, 20, 30, 40]  # broadcast (4,) -> (5,4)
X[1:4, 2:6] += 5

print(X)
assert (X[0, 2:6] == [10, 20, 30, 40]).all()
assert (X[1, 2:6] == [15, 25, 35, 45]).all()
assert (X[3, 2:6] == [15, 25, 35, 45]).all()
assert (X[4, 2:6] == [10, 20, 30, 40]).all()

[[ 0  0 10 20 30 40  0]
 [ 0  0 15 25 35 45  0]
 [ 0  0 15 25 35 45  0]
 [ 0  0 15 25 35 45  0]
 [ 0  0 10 20 30 40  0]]


### Exercise 6 — Overlapping slices (order matters)

**Problem**
Let `a = np.arange(10)`.

1. Do `a[1:8] = a[2:9]` (a slice assigned from another slice).
2. Predict the output, then verify by printing `a`.
3. Explain *by code evidence* whether NumPy behaves like it copies the RHS first.

> Hint: compare the result to doing it with an explicit `.copy()` on the RHS.

In [7]:
# --- Solution (Exercise 6) ---
# In NumPy, overlapping slice assignment is handled safely (as if RHS is copied first).

a1 = np.arange(10)
a1[1:8] = a1[2:9]

a2 = np.arange(10)
a2[1:8] = a2[2:9].copy()

print("overlap assignment:", a1)
print("explicit copy     :", a2)

assert np.array_equal(a1, a2)
# Expected: [0,2,3,4,5,6,7,8,8,9]

overlap assignment: [0 2 3 4 5 6 7 8 8 9]
explicit copy     : [0 2 3 4 5 6 7 8 8 9]


### Exercise 7 — Sliding windows + slicing

**Problem**
Given `s = np.arange(10)`, build a 2D view where each row is a length-4 sliding window.
Then keep only windows with even starting index (0,2,4,6) using slicing.

Use `sliding_window_view` and slicing (no loops).

In [8]:
# --- Solution (Exercise 7) ---
s = np.arange(10)
W = sliding_window_view(s, window_shape=4)   # shape (7,4)
W_even = W[::2]                              # starts at 0,2,4,6

print("s:\n", s)
print("\nW:\n", W)
print("\nW_even:\n", W_even)

assert W.shape == (7, 4)
assert W_even.shape == (4, 4)
assert (W_even[0] == [0, 1, 2, 3]).all()
assert (W_even[1] == [2, 3, 4, 5]).all()
assert np.shares_memory(s, W_even)

s:
 [0 1 2 3 4 5 6 7 8 9]

W:
 [[0 1 2 3]
 [1 2 3 4]
 [2 3 4 5]
 [3 4 5 6]
 [4 5 6 7]
 [5 6 7 8]
 [6 7 8 9]]

W_even:
 [[0 1 2 3]
 [2 3 4 5]
 [4 5 6 7]
 [6 7 8 9]]


### Exercise 8 — dtype pitfalls when assigning to slices

**Problem**
Create `u = np.array([250, 251, 252, 253], dtype=np.uint8)`.
1. Assign `u[1:3] = [300, -5]`.
2. Print `u` and `u.astype(int)` to explain what happened.
3. Show one safe approach: compute in a larger dtype and `clip` before storing back.

In [9]:
u = np.array([250, 251, 252, 253], dtype=np.uint8)
print("Before:", u, "dtype=", u.dtype)

# Version-robust way to show wrap-around: compute modulo 256 in a larger dtype, then cast
vals = np.array([300, -5], dtype=np.int64)
u[1:3] = (vals % 256).astype(np.uint8)

print("After :", u, "dtype=", u.dtype)
print("As int:", u.astype(int))

# Safe approach: compute in larger dtype, clip, then cast
u2 = np.array([250, 251, 252, 253], dtype=np.uint8)
tmp = u2.astype(np.int64)
tmp[1:3] = [300, -5]
tmp = np.clip(tmp, 0, 255)
u2[:] = tmp.astype(np.uint8)

print("\nClipped result:", u2)
assert (u2.astype(int) == [250, 255, 0, 253]).all()


Before: [250 251 252 253] dtype= uint8
After : [250  44 251 253] dtype= uint8
As int: [250  44 251 253]

Clipped result: [250 255   0 253]
